In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import geopandas as gpd
import pandas as pd
import dash_bootstrap_components as dbc
import numpy as np
import dash_table
import plotly.graph_objects as go
import folium
import branca.colormap as cm
import rasterio 
import matplotlib.cm
import rioxarray
import branca
from pyngrok import ngrok

In [24]:
# Load Data
amenity_exposure_lst = pd.read_csv('https://storage.googleapis.com/data_portal_exposure/data/indicators/amenity_exposure_heat.csv')
# filrer outliers
amenity_exposure_lst = amenity_exposure_lst[amenity_exposure_lst['exposure_lst_mean']>-10]
amenity_exposure_lst = amenity_exposure_lst.dropna(subset=['lst_dev_city_mean_exposure_class'])

# get city list
# available_cities = amenity_exposure_lst['city_name'].unique()
available_cities = ['Vitacura']
# get samenity sector list
available_amenity_sectors = amenity_exposure_lst['gcom_sector_name'].unique()
# create list of indicators
available_kpi = ['exposure_lst_mean',
                 'lst_value_dev_city_mean','lst_pecent_dev_city_mean','lst_dev_city_mean_exposure_class',
                 'lst_value_dev_city_perc_90','lst_pecent_dev_city_perc_90','lst_dev_city_perc90_exposure_class']
# create empty df
city_sector_amenity_exposure_col = pd.DataFrame(columns=['City name',
                                           'Sector Name',
                                           'Average LST by sector (°C)',
                                           'Min LST by sector (°C)',
                                           'Max LST by sector (°C)',
                                           'Number of amenities (OSM)',
                                           'Number of exposed amenities',
                                           'Number of not exposed amenities',
                                           'Percent of highly exposed amenities (%)'
                                           ])

# prepare raster layer
# seelect city id
# city_id = "MAR-Marrakech"
# # define path
# fpath = "https://storage.googleapis.com/urbanshift/lst/lst_Mean/lst-"+city_id+".tif"


# with rasterio.open(fpath_Belem) as src:
#     boundary = src.bounds
#     img_Belem = src.read()
fpath_Vitacura = 'https://storage.googleapis.com/data_portal_exposure/data/land_surface_temperature/lst_mean/lst_CHL-Vitacura.tif'

    
img_Vitacura = rioxarray.open_rasterio(fpath_Vitacura, masked=True)

def get_LST(city_name):
    if city_name == 'Vitacura':
        img = img_Vitacura
    return img



# color amenity based on indicators
colormap_step = branca.colormap.StepColormap(colors=['blue','red'],
                                          index=[0, 100, 200], 
                                          vmin=0, vmax=200,
                                          caption='Deviation percent from city average (%)')


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
#external_stylesheets = = [dbc.themes.BOOTSTRAP])

# specify color palettes for plots
BoldRiverBlue =  '#242456'
BoldSunYellow =  '#FFD450'
BoldGrassGreen =  '#2A553E'
BoldEarthGrey =  '#7B7A66'
BoldBrickOrange =  '#F26640'
LightRiverBlue =  '#E3E6FF'
LightSunYellow =  '#FFEDBA'
LightGrassGreen =  '#C4F4D5'
LightEarthGrey =  '#ECE2D6'
LightBrickOrange =  '#FED3CF'
White = '#FFFFFF'
Black = '#000000'

# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.title = 'Data Portal - City exposure'

app.layout = html.Div([
    html.H2("Amenity Exposure To Extreme Heat events"), 
    # html.H2("Exposure to High Land Surface Temperature"), 
    html.Div([
        html.Div([
            html.Div([
                # Define city filter
                html.Label([
                    "Select a city: ",
                    dcc.Dropdown(id='city',
                                 clearable=False,
                                 value='Vitacura',
                                 options=[{'label': i, 'value': i} for i in available_cities
                                         ])
                ]),
                html.Br(),
                # Define sector filter
                html.Label([
                    "Select a sector: ",
                    dcc.Dropdown(id='sector',
                                 value=available_amenity_sectors[0:8],
                                 multi=True,
                                 options=[{'label': i, 'value': i} for i in available_amenity_sectors
                                         ])
                ]),
                html.Br(),
                # Define indicator filter
                html.Label([
                    "Select an indicator: ",
                    dcc.Dropdown(id='kpi', 
                                 clearable=False,
                                 value='lst_pecent_dev_city_mean',
                                 options = [
                                     {'label': 'Percent of Deviation city average LST', 'value': 'lst_pecent_dev_city_mean'},
                                 ]
                                )
                ]),
                html.Br(),
                # Define xposed amenities filter
                html.Label([
                    "Filter exposed amenities: ",
                    dcc.Checklist(id = 'amenity_exposure',
                                  options=[
                                      {'label': 'Exposed amenities', 'value': 'Exposed'},
                                      {'label': 'Not Exposed amenities', 'value': 'Not Exposed'},
                                  ],
                                  value=['Exposed', 'Not Exposed']
                                 )
                    ]),
                html.Br(),
                # Main KPI average devation percent
                html.Div(id='kpi_city_avg',
                        style={
                            'font-weight': 'bold',
                            'font-size': '15px',
                            'color': '#FFFFFF',
                            'textAlign': 'center',
                            'backgroundColor':'#000080',# '#B0B0B0', 
                            'marginBottom': 0, 
                            'marginTop': 0,
                            "height": 25
                        }),
                html.P('Deviation from city average LST',
                      style={
                            'font-weight': 'bold',
                            'font-size': '17px',
                            'color': '#FFFFFF',
                            'textAlign': 'center',
                            'backgroundColor': '#00BFFF',
                            'marginBottom': 0, 
                            'marginTop': 10,
                            "height": 25
                        }),
                # Main KPI average devation percent
                html.Div(id='kpi_avg_deviation_percent',
                        style={
                            'font-weight': 'bold',
                            'font-size': '17px',
                            'color': '#FFFFFF',
                            'textAlign': 'center',
                            'backgroundColor':'#00BFFF',# '#B0B0B0', 
                            'marginBottom': 0, 
                            'marginTop': 0,
                            "height": 25
                        }),
                # Main KPI average devation value
                html.Div(id='kpi_avg_deviation_value',
                        style={
                            'font-weight': 'bold',
                            'font-size': '17px',
                            'color': '#FFFFFF',
                            'textAlign': 'center',
                            'backgroundColor': '#00BFFF',
                            'marginBottom': 10, 
                            'marginTop': 0,
                            "height": 25
                        }),
                # Define exposure threshold filter
                html.Label([
                    "Define LST threshold: ",
                    dcc.Slider(
                        id='slider_threshold',
                        min=20,
                        max=50,
                        step=5,
                        marks={
                            20: {'label': '20 °C', 'style': {'color': '#000080'}},
                            25: '25 °C',
                            30: '30 °C',
                            35: '35 °C',
                            40: '40 °C',
                            45: '45 °C',
                            50:  {'label': '50 °C', 'style': {'color': '#f50'}}
                        },
#                         value=30,
                    ),
                ]),
                # Define main KPI
#                 html.Div([
#                     dcc.Graph(id='fig_kpi')
#                 ]),
            ],
                className="three columns",
            ),
            # plot map
            html.Div([
#                 html.Iframe(id='map',
#                             srcDoc=None,
#                             width='100%',
#                             height='600',
#                             hidden=False),
                html.Iframe(id='map_threshold', 
                    srcDoc=None, 
                    width='100%', 
                    height='600', 
                    hidden=False),
            ],
                className="nine columns",
            ),
        ],
            className="row"),
#         html.Iframe(id='map_threshold', 
#                     srcDoc=None, 
#                     width='100%', 
#                     height='600', 
#                     hidden=False),
#         html.Div([
#             html.Div([
#                 dcc.Graph(id='fig_kpi_city_avg'),
#             ],
#                 className="three columns"),
#             html.Div([
#                 dcc.Graph(id='fig_kpi_city_90perc'),
#             ],
#                 className="three columns"),
#             html.Div([
#                 dcc.Graph(id='fig_kpi_amenity_avg'),
#             ],
#                 className="three columns"),
#             html.Div([
#                 dcc.Graph(id='fig_kpi_amenity_90perc'),
#             ],
#                 className="three columns"),
#         ],
#             style = {'height': '200'},
#             className="row",
#         ),
        html.Div([
            html.Div([
                dcc.Graph(id='graph-sector'),
            ],
                className="six columns"),
            html.Div([
                dcc.Graph(id='graph-sector-boxplot'),
            ],
                className="six columns"),
        ],
            className="row",
        ),
        html.Div(
            dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in city_sector_amenity_exposure_col.columns],
                page_size=10,
                sort_action="native",
                style_cell={
                    'textAlign': 'left',
                    'backgroundColor': White,
                    'color': Black,
                    'whiteSpace': 'normal',
                    'height': 'auto'
                },
                style_header={
                    'backgroundColor': '#00BFFF',
                    'fontWeight': 'bold',
                    'color': 'white'
                },
            ),
            className="twelve columns"
        ),
    ]),
])

                       
# Define callback to update graph
@app.callback(
    [
#         Output('map', 'srcDoc'), 
        Output('kpi_city_avg', 'children'),
        Output('kpi_avg_deviation_percent', 'children'),
        Output('kpi_avg_deviation_value', 'children'),
        Output('map_threshold', 'srcDoc'),
        Output('graph-sector', 'figure'),
        Output('graph-sector-boxplot', 'figure'),
        Output('table', 'data'),
#         Output('fig_kpi', 'figure'),
#         Output('fig_kpi_city_avg', 'figure'),
#         Output('fig_kpi_city_90perc', 'figure'),
#         Output('fig_kpi_amenity_avg', 'figure'),
#         Output('fig_kpi_amenity_90perc', 'figure'),
    ],
    [
        Input("city", "value"),
        Input("sector", "value"),
        Input("kpi", "value"),
        Input("amenity_exposure", "value"),
        Input("slider_threshold", "value")
    ]
)

def update_map(city_name, sector_name, kpi, amenity_exposure, slider_threshold):
    
    # filter city data
    amenity_exposure_lst_selected = amenity_exposure_lst[amenity_exposure_lst['city_name'] == city_name]
    amenity_exposure_lst_selected_avg = amenity_exposure_lst_selected.exposure_lst_mean.mean()
    # avreage heat witnin amenities
    amenity_exposure_lst_selected_city_avg = amenity_exposure_lst_selected.exposure_lst_mean.mean()

    # city main kpi
    city_lst_avg = amenity_exposure_lst_selected.city_lst_avg.mean()
    city_lst_90_perc = amenity_exposure_lst_selected.city_lst_perc_90.mean()
    


    # amenities main kpi avg threshold
    amenity_lst_value_dev_city_mean = round(amenity_exposure_lst_selected.lst_value_dev_city_mean.mean(),2)
    amenity_lst_pecent_dev_city_mean= round(amenity_exposure_lst_selected.lst_pecent_dev_city_mean.mean(),2)

    # amenities main kpi 90th percentile threshold
    amenity_lst_value_dev_city_90perc = round(amenity_exposure_lst_selected.lst_value_dev_city_perc_90.mean(),2)
    amenity_lst_pecent_dev_city_90perc = round(amenity_exposure_lst_selected.lst_pecent_dev_city_perc_90.mean(),2)


    #  filter based on sectors
    amenity_exposure_lst_selected = amenity_exposure_lst_selected[amenity_exposure_lst_selected['gcom_sector_name'].isin(sector_name)]

    # average heat within sector selected amenities
    amenity_exposure_lst_selected_city_sector_avg = amenity_exposure_lst_selected.exposure_lst_mean.mean()
    
    # test filtering by sector
    amenity_lst_value_dev_city_mean = round(amenity_exposure_lst_selected.lst_value_dev_city_mean.mean(),2)
    amenity_lst_pecent_dev_city_mean= round(amenity_exposure_lst_selected.lst_pecent_dev_city_mean.mean(),2)
    
    #  filter based on exposure class
    amenity_exposure_lst_selected = amenity_exposure_lst_selected[amenity_exposure_lst_selected['lst_dev_city_mean_exposure_class'].isin(amenity_exposure)]
    

    #----------------------------
    # generate folium exposure map
    #----------------------------


    img = get_LST(city_name = city_name)
    
    lon = img.x.values.astype(np.float64)
    lon_mean = lon.mean()
    lat = img.y.values.astype(np.float64)
    lat_mean = lat.mean()


    data2plot = img[0]
    normed_data = (data2plot - np.nanmin(data2plot)) / (np.nanmax(data2plot) - np.nanmin(data2plot))
    cm = matplotlib.cm.get_cmap('coolwarm') 
    colored_data = cm(normed_data) 

#     m = folium.Map(location=[lat_mean, lon_mean],zoom_start=10)

#     image = folium.raster_layers.ImageOverlay(
#         name="Land Surface Temperature",
#         opacity=.8,
#         image = colored_data,
#         bounds=[[lat.min(), lon.min()], [lat.max(), lon.max()]])
    
#     image.add_to(m)
    
#     # plot amenities of the selected cityfor i in range(0,len(amenity_exposure_lst_selected)):
#     for i in range(0,len(amenity_exposure_lst_selected)):
#         folium.CircleMarker(
#             location=[amenity_exposure_lst_selected.iloc[i]['latitude'], amenity_exposure_lst_selected.iloc[i]['longitude']],
#             popup=amenity_exposure_lst_selected.iloc[i][kpi],
#             color = colormap_step(amenity_exposure_lst_selected.iloc[i][kpi]),
#             radius=2,
#             fill=True,
#           name = "amenities",
#        ).add_to(m)

#     m.add_child(colormap_step)
    
#     folium.LayerControl().add_to(m)

#     m.save('map_folium.html')
    
    #-----------------------------------------------------
    # generate folium exposure map based on slider threshold
    #-----------------------------------------------------
    
    
    # compute deviation based on selected threshold
    amenity_exposure_lst_selected['lst_pecent_dev_threshold'] = round((amenity_exposure_lst_selected['exposure_lst_mean'] * 100)/slider_threshold,2)
    # attribute exposure class based on selected threshold
    amenity_exposure_lst_selected.loc[amenity_exposure_lst_selected['lst_pecent_dev_threshold'] >= 100, 'lst_dev_threshold_exposure_class'] = 'Exposed'
    amenity_exposure_lst_selected.loc[amenity_exposure_lst_selected['lst_pecent_dev_threshold'] < 100, 'lst_dev_threshold_exposure_class'] = 'Not Exposed'
    
    # plot map
    map_threshold = folium.Map(location=[lat_mean, lon_mean],zoom_start=10)
    
    image = folium.raster_layers.ImageOverlay(
        name="Land Surface Temperature",
        opacity=.8,
        image = colored_data,
        bounds=[[lat.min(), lon.min()], [lat.max(), lon.max()]])
    
    # add raster layer
    image.add_to(map_threshold)
    
    # plot amenities of the selected city
    for i in range(0,len(amenity_exposure_lst_selected)):
        folium.CircleMarker(location=[amenity_exposure_lst_selected.iloc[i]['latitude'], amenity_exposure_lst_selected.iloc[i]['longitude']],
                            popup=amenity_exposure_lst_selected.iloc[i]['lst_pecent_dev_threshold'],
                            color = colormap_step(amenity_exposure_lst_selected.iloc[i]['lst_pecent_dev_threshold']),
                            radius=2,
                            fill=True,
                            name = "amenities").add_to(map_threshold)

    map_threshold.add_child(colormap_step)
    
    folium.LayerControl().add_to(map_threshold)

    map_threshold.save('map_threshold.html')
    
    
    #----------------------------
    # generate sector comparison figures
    #----------------------------

    # prepare sector comparison dataframe
    city_sector_amenity_exposure = (amenity_exposure_lst_selected 
                            .groupby(["city_name","gcom_sector_name"])
                            .agg(nb_amenity = pd.NamedAgg(column='feature_value', aggfunc='count'),
                                 nb_amenity_exposed=pd.NamedAgg(column='exposure_lst_mean', aggfunc=lambda x: (x > amenity_exposure_lst_selected_avg).sum()), 
                                 nb_amenity_not_exposed=pd.NamedAgg(column='exposure_lst_mean', aggfunc=lambda x: (x < amenity_exposure_lst_selected_avg).sum()),
                                 avg_temp = pd.NamedAgg(column='exposure_lst_mean', aggfunc=np.mean),
                                 min_temp = pd.NamedAgg(column='exposure_lst_mean', aggfunc='min'),
                                 max_temp = pd.NamedAgg(column='exposure_lst_mean', aggfunc='max')
                                )
                            .reset_index()
                            )
  
    # city_sector_amenity_exposure['percent_amenity_exposed'] = round(city_sector_amenity_exposure['nb_amenity_exposed']/(city_sector_amenity_exposure['nb_amenity_exposed'] + city_sector_amenity_exposure['nb_amenity_not_exposed']),2)

    city_sector_amenity_exposure = city_sector_amenity_exposure.round({'avg_temp': 2, 
                                                                   'min_temp': 2,
                                                                   'max_temp': 2})
  
    city_sector_amenity_exposure['percent_amenity_exposed'] = round(city_sector_amenity_exposure['nb_amenity_exposed']/(city_sector_amenity_exposure['nb_amenity_exposed'] + city_sector_amenity_exposure['nb_amenity_not_exposed']),4)*100

  
    # plot sector exposure
    fig_sectors = px.bar(
        city_sector_amenity_exposure.sort_values(by=['percent_amenity_exposed'],ascending=False),
        x='gcom_sector_name',
        y='percent_amenity_exposed',
        color_discrete_sequence=['#00BFFF'],
        hover_data=['city_name', 'gcom_sector_name', "percent_amenity_exposed"],
        hover_name="gcom_sector_name",
        labels={
            'city_name':'City name',
            'gcom_sector_name': 'Sector',
            'percent_amenity_exposed': 'Percent of exposed amenities'
        },
        title="Amenity exposure by sectors <br><sup>(Amenities with LST values higher than average city values)</sup>",)
  
    # plot boxplot
    fig_sectors_boxplot = px.box(amenity_exposure_lst_selected,
                                 x="gcom_sector_name",
                                 y="exposure_lst_mean",
                                 labels={
                                     "exposure_lst_mean": "Amenity Land Surface Temperature",
                                     "gcom_sector_name": "Sectors",
                                 },
                                 title = "Average LST values distribution by sectors")

    fig_sectors_boxplot.update_traces(showlegend=False)
  
    # display table

    city_sector_amenity_exposure_table = (city_sector_amenity_exposure
                                          .filter(['city_name',
                                                   'gcom_sector_name',
                                                   'avg_temp',
                                                   'min_temp',
                                                   'max_temp',
                                                   'nb_amenity',
                                                   'nb_amenity_exposed',
                                                   'nb_amenity_not_exposed',
                                                   'percent_amenity_exposed'])
                                          .rename(columns={'city_name': 'City name',
                                                           'gcom_sector_name': 'Sector Name',
                                                           'avg_temp': 'Average LST by sector (°C)',
                                                           'min_temp': 'Min LST by sector (°C)',
                                                           'max_temp': 'Max LST by sector (°C)',
                                                           'nb_amenity': 'Number of amenities (OSM)',
                                                           'nb_amenity_exposed': 'Number of exposed amenities',
                                                           'nb_amenity_not_exposed': 'Number of not exposed amenities',
                                                           'percent_amenity_exposed': 'Percent of highly exposed amenities (%)'})
                                          .reset_index())
  
    city_lst_avg = float(amenity_exposure_lst_selected.city_lst_avg.unique())
  
    ########################## 
    # display Main kpi
    ########################## 
    
    # City Average heat
    kpi_city_lst_avg = round(int(city_lst_avg))
    
    # amenity deviation heat from city average
    kpi_avg_deviation_percent = round(int(amenity_lst_pecent_dev_city_mean))
    
    #kpi_avg_deviation_value = amenity_exposure_lst_selected_city_sector_avg - city_lst_avg
    #kpi_avg_deviation_value = round(int(kpi_avg_deviation_value))
    
    # amenity deviation heat from all amenities
    kpi_avg_deviation_value = amenity_exposure_lst_selected_city_sector_avg - amenity_exposure_lst_selected_city_avg
    kpi_avg_deviation_value = round(kpi_avg_deviation_value,2)
    
    
    return 'City average LST (°C): {}'.format(kpi_city_lst_avg),'(%): {}'.format(kpi_avg_deviation_percent),'(°C): {}'.format(kpi_avg_deviation_value), open('map_threshold.html', 'r').read(), fig_sectors, fig_sectors_boxplot, city_sector_amenity_exposure_table.to_dict('records')


@app.callback(
    Output("slider_threshold", "value"),
    [Input("city", "value")]
)
def populate_threshold_slider(city_name):
    amenity_exposure_lst_selected = amenity_exposure_lst[amenity_exposure_lst['city_name'] == city_name]
    amenity_exposure_lst_selected_avg = amenity_exposure_lst_selected.exposure_lst_mean.mean()
#     min = round(int(df[color].min())),
#     max = round(int(df[color].max())),
#     step = 0.5,
    value = amenity_exposure_lst_selected_avg
    return value 




# Run app and display result inline in the notebook
# app.run_server(mode='inline')
app.run_server(mode='external', debug=True, port=8051)
# if __name__ == '__main__':
#     app.run_server(debug=True)





Dash app running on http://127.0.0.1:8051/


C:\Users\Saif.Shabou\Anaconda3\envs\gee\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



# Deploy with ngrok

In [3]:
!ngrok authtoken 1x8VJqIAxHvjxJNfAoiZz6ujyna_2Atmbj2VhBR7gemdMVj4p

Authtoken saved to configuration file: C:\Users\Saif.Shabou/.ngrok2/ngrok.yml


In [4]:
ngrok.connect(8052)

<NgrokTunnel: "http://5ca7-2a01-cb08-23d-1e00-985a-4962-c50e-b933.ngrok.io" -> "http://localhost:8052">

t=2021-09-07T11:24:51+0200 lvl=eror msg="session closed, starting reconnect loop" obj=csess id=1b4b08530ae9 err="read tcp [2a01:cb08:23d:1e00:985a:4962:c50e:b933]:55266->[2600:1f16:d83:1201::6e74:1]:443: wsarecv: An established connection was aborted by the software in your host machine."
t=2021-09-07T11:25:01+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="dial tcp: i/o timeout"
t=2021-09-07T11:25:05+0200 lvl=eror msg="heartbeat timeout, terminating session" obj=csess id=5ab06972d828 clientid=4bc9c079e16aad6e1f5bcece76d8f821
t=2021-09-07T11:25:11+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="dial tcp: i/o timeout"
t=2021-09-07T11:25:12+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="Get \"https://dns.google.com/resolve?cd=true&name=tunnel.us.ngrok.com&type=A\": dial tcp: lookup dns.google.com: no such host"
t=2021-09-07T11:25:13+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b

t=2021-09-07T11:42:56+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="resolved tunnel.us.ngrok.com has no records"
t=2021-09-07T11:43:26+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="dial tcp: lookup tunnel.us.ngrok.com: no such host"
t=2021-09-07T11:43:56+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="Get \"https://dns.google.com/resolve?cd=true&name=tunnel.us.ngrok.com&type=A\": dial tcp: lookup dns.google.com: no such host"
t=2021-09-07T11:44:26+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="Get \"https://s3.amazonaws.com/dns.ngrok.com/tunnel.json\": dial tcp: lookup s3.amazonaws.com: no such host"
t=2021-09-07T11:44:56+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="resolved tunnel.us.ngrok.com has no records"
t=2021-09-07T11:45:26+0200 lvl=eror msg="failed to reconnect session" obj=csess id=1b4b08530ae9 err="dial tcp: lookup

In [10]:
ngrok.kill()

# Test

In [4]:
# Load Data
amenity_exposure_lst = pd.read_csv('https://storage.googleapis.com/data_portal_exposure/data/indicators/amenity_exposure_heat.csv')


In [12]:
city_name = 'Vitacura'

In [20]:
amenity_exposure_lst.head()

,Unnamed: 0,city_name,city_lst_avg,city_lst_perc_90,feature_key,feature_value,feature_category,latitude,longitude,geometry,...,lst_pecent_dev_city_perc_90,lst_dev_city_perc90_exposure_class,featureCategory,featureType,id,cityName,integrationDate,objectType,projectName,country_iso3
0,0,Vitacura,32.50379,35.864247,amenity,pharmacy,Healthcare,-33.376238,-70.525053,POINT (-70.5250533 -33.376238),...,90.79,Not Exposed,amenity,pharmacy,240434655.0,Vitacura,2021-09-23,amenity,dataportal,CHL
1,1,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.393896,-70.600037,POINT (-70.6000366 -33.3938961),...,90.54,Not Exposed,amenity,parking,253242064.0,Vitacura,2021-09-23,amenity,dataportal,CHL
2,2,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.402910,-70.580557,POINT (-70.5805565 -33.4029103),...,87.83,Not Exposed,amenity,parking,253272975.0,Vitacura,2021-09-23,amenity,dataportal,CHL
3,3,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.392850,-70.575862,POINT (-70.5758615 -33.3928496),...,92.89,Not Exposed,amenity,parking,253276941.0,Vitacura,2021-09-23,amenity,dataportal,CHL
4,4,Vitacura,32.50379,35.864247,amenity,parking,Transportation,-33.403386,-70.579463,POINT (-70.57946269999999 -33.4033863),...,87.44,Not Exposed,amenity,parking,253433629.0,Vitacura,2021-09-23,amenity,dataportal,CHL


In [13]:
amenity_exposure_lst_selected = amenity_exposure_lst[amenity_exposure_lst['city_name'] == city_name]
amenity_exposure_lst_selected_avg = amenity_exposure_lst_selected.exposure_lst_mean.mean()

In [14]:
amenity_exposure_lst_selected_avg

32.713971536061

In [15]:
# city main kpi
city_lst_avg = amenity_exposure_lst_selected.city_lst_avg.mean()
city_lst_90_perc = amenity_exposure_lst_selected.city_lst_perc_90.mean()
city_lst_avg

32.503790256644685

In [17]:
# amenities main kpi avg threshold
amenity_lst_value_dev_city_mean = round(amenity_exposure_lst_selected.lst_value_dev_city_mean.mean(),2)
amenity_lst_pecent_dev_city_mean= round(amenity_exposure_lst_selected.lst_pecent_dev_city_mean.mean(),2)
amenity_lst_value_dev_city_mean


0.21

In [ ]:
# amenities main kpi 90th percentile threshold
amenity_lst_value_dev_city_90perc = round(amenity_exposure_lst_selected.lst_value_dev_city_perc_90.mean(),2)
amenity_lst_pecent_dev_city_90perc = round(amenity_exposure_lst_selected.lst_pecent_dev_city_perc_90.mean(),2)